# SupplyChain class

In [1]:
from climada.util.api_client import Client
from climada_petals.engine import SupplyChain
from climada.entity import ImpfTropCyclone, ImpactFuncSet
from climada.engine.impact_calc import ImpactCalc

client = Client()

This tutorial shows how to use the `SupplyChain` class of CLIMADA. This class allows assessing indirect impacts via Input-Output modeling. Before diving into this class, it is highly recommended the user first understands how direct impact is calculated with CLIMADA by getting familiar with the `Exposures`, `Hazard` and `Impact` classes as the `SupplyChain` class runs on outputs from such analysis.

This tutorial shows how to set up a global supply chain risk analysis due to tropical cyclone that hit the United States.

# 1. Calculate direct economic impacts

As a first step, we load LitPop exposure for the USA from CLIMADA Data API.

In [2]:
exp_usa = client.get_litpop('USA')

https://climada.ethz.ch/data-api/v1/dataset	country_name=United States	data_type=litpop	exponents=(1,1)	limit=100000	name=None	status=active	version=None
2023-04-19 16:42:42,523 - climada.entity.exposures.base - INFO - Reading /Users/aciullo/climada/data/exposures/litpop/LitPop_150arcsec_USA/v2/LitPop_150arcsec_USA.hdf5


Then, we load a probabilistic tropical cyclone events set for the USA relative to current climate from the CLIMADA Data API.

In [3]:
tc_usa = client.get_hazard('tropical_cyclone', properties={'country_iso3alpha':'USA', 'climate_scenario':'historical'})

https://climada.ethz.ch/data-api/v1/dataset	climate_scenario=historical	country_iso3alpha=USA	data_type=tropical_cyclone	limit=100000	name=None	status=active	version=None
2023-04-19 16:43:00,758 - climada.hazard.base - INFO - Reading /Users/aciullo/climada/data/hazard/tropical_cyclone/tropical_cyclone_10synth_tracks_150arcsec_USA_1980_2020/v2/tropical_cyclone_10synth_tracks_150arcsec_USA_1980_2020.hdf5


/Users/aciullo/opt/anaconda3/envs/climada_env/lib/python3.8/site-packages/geopandas/geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)
/Users/aciullo/opt/anaconda3/envs/climada_env/lib/python3.8/site-packages/geopandas/geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)


We finally calculate impact.

In [4]:
# Define impact function
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_set = ImpactFuncSet()
impf_set.append(impf_tc)
impf_set.check()

In [5]:
# Calculate direct impacts to the USA due to TC
imp_calc = ImpactCalc(exp_usa, impf_set, tc_usa)
direct_impact_usa = imp_calc.impact()

2023-04-19 16:43:00,966 - climada.entity.exposures.base - INFO - No specific impact function column found for hazard TC. Using the anonymous 'impf_' column.
2023-04-19 16:43:00,966 - climada.entity.exposures.base - INFO - Matching 643099 exposures with 503715 centroids.
2023-04-19 16:43:00,976 - climada.util.coordinates - INFO - No exact centroid match found. Reprojecting coordinates to nearest neighbor closer than the threshold = 100
2023-04-19 16:43:01,937 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 129140 coordinates.
2023-04-19 16:43:01,968 - climada.engine.impact_calc - INFO - Calculating impact for 1327884 assets (>0) and 43560 events.


# 2. Calculate indirect economic impacts

## 2.1 Instantiate a `SupplyChain` object by loading the Multi-Regional Input-Output Table of interest.

`SupplyChain` computes indirect economic impacts via Input-Output modeling. At the core of IO modeling lies an Input-Output Table and `SupplyChain` uses the [pymrio](https://pymrio.readthedocs.io/en/latest/) python package to download, parse and save several Multi-Regional Input Output Tables (MRIOTs). The following ones can be automatically downloaded:
- [EXIOBASE3](https://zenodo.org/record/3583071#.Y_4zZuzMK3I) (1995-2011; 44 countries; 163 industries)​
- [WIOD16](https://www.rug.nl/ggdc/valuechain/wiod/) (2000-2014; 43 countries; 56 industries)​
- [OECD21](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm) (1995-2018; 66 countries; 45 industries)​

The first step is to instantiate a `SupplyChain` class. When using one the supported MRIOTs, this can done by calling the `from_mriot` class method.

In [6]:
supchain = SupplyChain.from_mriot(mriot_type='WIOD16', mriot_year=2011)

The instatiated class now has an 'mriot' attribute, which is a pymrio 'IOSystem' object. As such, one can easily access several info of the MRIOT incl. regions, sectors, total production, transaction matrix and final demand.

In [7]:
# regions
supchain.mriot.get_regions()

Index(['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP', 'CZE',
       'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN',
       'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX',
       'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 'SWE',
       'TUR', 'TWN', 'USA', 'ROW'],
      dtype='object', name='region')

In [8]:
# sectors
supchain.mriot.get_sectors()

Index(['Crop and animal production, hunting and related service activities',
       'Forestry and logging', 'Fishing and aquaculture',
       'Mining and quarrying',
       'Manufacture of food products, beverages and tobacco products',
       'Manufacture of textiles, wearing apparel and leather products',
       'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials',
       'Manufacture of paper and paper products',
       'Printing and reproduction of recorded media',
       'Manufacture of coke and refined petroleum products ',
       'Manufacture of chemicals and chemical products ',
       'Manufacture of basic pharmaceutical products and pharmaceutical preparations',
       'Manufacture of rubber and plastic products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of basic metals',
       'Manufacture of fabricated metal products, except machinery and equipment',
       

In [9]:
# transaction matrix
supchain.mriot.Z

region                                                                                                                   AUS  \
sector                                                    Crop and animal production, hunting and related service activities   
region sector                                                                                                                  
AUS    Crop and animal production, hunting and related...                                       10954.209508                   
       Forestry and logging                                                                       113.905445                   
       Fishing and aquaculture                                                                     22.267974                   
       Mining and quarrying                                                                       232.437685                   
       Manufacture of food products, beverages and tob...                                        1662.225516                   
...                                                                                                      ...                   
ROW    Education                                                                                    1.305193                   
       Human health and social work activities                                                      0.535337                   
       Other service activities                                                                     1.856064                   
       Activities of households as employers; undiffer...                                           0.027816                   
       Activities of extraterritorial organizations an...                                           0.000000                   

region                                                                          \
sector                                                    Forestry and logging   
region sector                                                                    
AUS    Crop and animal production, hunting and related...           337.318807   
       Forestry and logging                                         127.867760   
       Fishing and aquaculture                                        0.025312   
       Mining and quarrying                                           2.323883   
       Manufacture of food products, beverages and tob...             4.046841   
...                                                                        ...   
ROW    Education                                                      0.022124   
       Human health and social work activities                        0.038688   
       Other service activities                                       0.058495   
       Activities of households as employers; undiffer...             0.000064   
       Activities of extraterritorial organizations an...             0.000000   

region                                                                             \
sector                                                    Fishing and aquaculture   
region sector                                                                       
AUS    Crop and animal production, hunting and related...              166.404066   
       Forestry and logging                                              0.588653   
       Fishing and aquaculture                                          21.871529   
       Mining and quarrying                                              8.353050   
       Manufacture of food products, beverages and tob...               78.987732   
...                                                                           ...   
ROW    Education                                                         0.093113   
       Human health and social work activities                           0.141195   
       Other service activities                                          0.149588   
       Activities of households as emp

In [10]:
# final demand
supchain.mriot.Y

final demand
region sector                                                           
AUS    Crop and animal production, hunting and related...   14467.050267
       Forestry and logging                                   342.352502
       Fishing and aquaculture                               1787.071582
       Mining and quarrying                                 19445.035584
       Manufacture of food products, beverages and tob...   51480.063474
...                                                                  ...
ROW    Education                                           483744.763934
       Human health and social work activities             382325.468496
       Other service activities                            234071.579585
       Activities of households as employers; undiffer...   14600.614007
       Activities of extraterritorial organizations an...      33.142482

[2464 rows x 1 columns]

In [11]:
# total production
supchain.mriot.x

total production
region sector                                                              
AUS    Crop and animal production, hunting and related...      69566.630223
       Forestry and logging                                     2719.293635
       Fishing and aquaculture                                  3113.703184
       Mining and quarrying                                   250207.439237
       Manufacture of food products, beverages and tob...      92033.592425
...                                                                     ...
ROW    Education                                              508147.713351
       Human health and social work activities                399362.733830
       Other service activities                               367396.468789
       Activities of households as employers; undiffer...      22003.013824
       Activities of extraterritorial organizations an...         33.142482

[2464 rows x 1 columns]

## 2.2 Assign exposure and stock impact to MRIOT country-sector

Direct economic impacts are calculated at the exposure level, while the indirect impacts will be calculated at the countries-sectors level specified by the MRIOT. Thus, we first need to assign direct impacts to countries-sectors levels. 

The first step is to identify what sectors in the MRIOT can be represented by the used exposure. In this case we use a `LitPop` exposure which is a proxy of the service sectors. Then, one needs to identify what subsectors in the MRIOT belong to the identified sector. In the case of WIOD16, we identify sebsectors ranging from position 26 to 56 as belonging to the service sector. Finally, the total exposure and direct impact are assigned to each subsector proportionally to their contribution to the sector's total production. This can be calculated with the method `calc_secs_exp_imp_shock`.

In [12]:
impacted_secs = supchain.mriot.get_sectors()[range(26,56)].tolist()

In [13]:
supchain.calc_secs_exp_imp_shock(exp_usa, direct_impact_usa, impacted_secs)

/Users/aciullo/Documents/GitHub/climada_petals/climada_petals/engine/supplychain.py:450: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.secs_stock_exp.loc[:, (mriot_reg_name, impacted_secs)] += (
/Users/aciullo/Documents/GitHub/climada_petals/climada_petals/engine/supplychain.py:453: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.secs_stock_imp.loc[:, (mriot_reg_name, impacted_secs)] += (


This creates the attributes `supchain.secs_stock_imp`, `supchain.secs_stock_exp`, which are `Impact` and `Exposure` disaggregated at the country-sector level of the selected MRIOT, and `self.secs_stock_shock`, which is the ratio of the two.

This direct damages to assets/stocks shall now be transformed into direct production losses. By default, this is done by assuming that the fraction of lost production is the same as the fraction of lost stock. Since the latter is stored in `self.secs_stock_shock`, one can multiply these fractions by the sectorial total production derived by the MRIOT to calculate direct production losses. This is done via `calc_direct_production_impacts`.

Such a default behaviour can be changed, and the fraction of lost production can be set to be higher or lower the fraction of lost stock. This can be achieved by passing the `stock_to_prod_shock` to `calc_direct_production_impacts`. It is recommended to do so only when detailed knowledge about the relationship between stock and production losses is available.

## 2.3 Calculate local production losses

In [14]:
supchain.calc_direct_production_impacts()

This creates the following outputs:

In [15]:
supchain.dir_prod_impt_mat

region                                                                AUS  \
sector Crop and animal production, hunting and related service activities   
1                                                     0.0                   
2                                                     0.0                   
3                                                     0.0                   
4                                                     0.0                   
5                                                     0.0                   
...                                                   ...                   
43556                                                 0.0                   
43557                                                 0.0                   
43558                                                 0.0                   
43559                                                 0.0                   
43560                                                 0.0                   

region                                                                    \
sector Forestry and logging Fishing and aquaculture Mining and quarrying   
1                       0.0                     0.0                  0.0   
2                       0.0                     0.0                  0.0   
3                       0.0                     0.0                  0.0   
4                       0.0                     0.0                  0.0   
5                       0.0                     0.0                  0.0   
...                     ...                     ...                  ...   
43556                   0.0                     0.0                  0.0   
43557                   0.0                     0.0                  0.0   
43558                   0.0                     0.0                  0.0   
43559                   0.0                     0.0                  0.0   
43560                   0.0                     0.0                  0.0   

region                                                               \
sector Manufacture of food products, beverages and tobacco products   
1                                                     0.0             
2                                                     0.0             
3                                                     0.0             
4                                                     0.0             
5                                                     0.0             
...                                                   ...             
43556                                                 0.0             
43557                                                 0.0             
43558                                                 0.0             
43559                                                 0.0             
43560                                                 0.0             

region                                                                \
sector Manufacture of textiles, wearing apparel and leather products   
1                                                     0.0              
2                                                     0.0              
3                                                     0.0              
4                                                     0.0              
5                                                     0.0              
...                                                   ...              
43556                                                 0.0              
43557                                                 0.0              
43558                                                 0.0              
43559                                                 0.0              
43560                                                 0.0              

region                                                                                                                                  \
sector Man

which is a `pandas.DataFrame` with indexes equal to the hazard events id and columns that represent the country-sectors in the MRIOT. This local production loss is obviously located only in the USA and only affect the sectors that are representative of the used exposure:

In [16]:
print(supchain.dir_prod_impt_mat.loc[:,('USA', impacted_secs)].sum().sum(),
      supchain.dir_prod_impt_mat.sum().sum())

5173005962729.039 5173005962729.039


which is a `pandas.Series` of annual expected production losses for the country-sectors defined in the MRIOT.

## 2.4 Calculate the propagation of production losses

In order to calculate how these local production losses propagate through the supply chain, one can use the `calc_indirect_production_impacts` method. This calculation can be executed following the `ghosh` or the `leontief` methods.

In [17]:
supchain.calc_indirect_production_impacts(direct_impact_usa.event_id, io_approach='ghosh')

This creates the following outputs:

In [18]:
supchain.indir_prod_impt_mat

region                                                                AUS  \
sector Crop and animal production, hunting and related service activities   
1                                                     0.0                   
2                                                     0.0                   
3                                                     0.0                   
4                                                     0.0                   
5                                                     0.0                   
...                                                   ...                   
43556                                                 0.0                   
43557                                                 0.0                   
43558                                                 0.0                   
43559                                                 0.0                   
43560                                                 0.0                   

region                                                                    \
sector Forestry and logging Fishing and aquaculture Mining and quarrying   
1                       0.0                     0.0                  0.0   
2                       0.0                     0.0                  0.0   
3                       0.0                     0.0                  0.0   
4                       0.0                     0.0                  0.0   
5                       0.0                     0.0                  0.0   
...                     ...                     ...                  ...   
43556                   0.0                     0.0                  0.0   
43557                   0.0                     0.0                  0.0   
43558                   0.0                     0.0                  0.0   
43559                   0.0                     0.0                  0.0   
43560                   0.0                     0.0                  0.0   

region                                                               \
sector Manufacture of food products, beverages and tobacco products   
1                                                     0.0             
2                                                     0.0             
3                                                     0.0             
4                                                     0.0             
5                                                     0.0             
...                                                   ...             
43556                                                 0.0             
43557                                                 0.0             
43558                                                 0.0             
43559                                                 0.0             
43560                                                 0.0             

region                                                                \
sector Manufacture of textiles, wearing apparel and leather products   
1                                                     0.0              
2                                                     0.0              
3                                                     0.0              
4                                                     0.0              
5                                                     0.0              
...                                                   ...              
43556                                                 0.0              
43557                                                 0.0              
43558                                                 0.0              
43559                                                 0.0              
43560                                                 0.0              

region                                                                                                                                  \
sector Man

This time we don't only have impacts to the USA, but also to other countries, e.g. Switzerland.

In [19]:
supchain.indir_prod_impt_mat.loc[:,('CHE', slice(None))].sum().sum()

3430485269.41916

## 2.5 Calculate total production losses

Finally, total impacts can be calculated via the `calc_total_production_impacts` method.

In [20]:
supchain.calc_total_production_impacts()

This creates the following outputs:

In [21]:
supchain.tot_prod_impt_mat

region                                                                AUS  \
sector Crop and animal production, hunting and related service activities   
1                                                     0.0                   
2                                                     0.0                   
3                                                     0.0                   
4                                                     0.0                   
5                                                     0.0                   
...                                                   ...                   
43556                                                 0.0                   
43557                                                 0.0                   
43558                                                 0.0                   
43559                                                 0.0                   
43560                                                 0.0                   

region                                                                    \
sector Forestry and logging Fishing and aquaculture Mining and quarrying   
1                       0.0                     0.0                  0.0   
2                       0.0                     0.0                  0.0   
3                       0.0                     0.0                  0.0   
4                       0.0                     0.0                  0.0   
5                       0.0                     0.0                  0.0   
...                     ...                     ...                  ...   
43556                   0.0                     0.0                  0.0   
43557                   0.0                     0.0                  0.0   
43558                   0.0                     0.0                  0.0   
43559                   0.0                     0.0                  0.0   
43560                   0.0                     0.0                  0.0   

region                                                               \
sector Manufacture of food products, beverages and tobacco products   
1                                                     0.0             
2                                                     0.0             
3                                                     0.0             
4                                                     0.0             
5                                                     0.0             
...                                                   ...             
43556                                                 0.0             
43557                                                 0.0             
43558                                                 0.0             
43559                                                 0.0             
43560                                                 0.0             

region                                                                \
sector Manufacture of textiles, wearing apparel and leather products   
1                                                     0.0              
2                                                     0.0              
3                                                     0.0              
4                                                     0.0              
5                                                     0.0              
...                                                   ...              
43556                                                 0.0              
43557                                                 0.0              
43558                                                 0.0              
43559                                                 0.0              
43560                                                 0.0              

region                                                                                                                                  \
sector Man

## Calculate all production losses in one step

Steps from 2.2 to 2.5 can be calculate all at once via the `calc_production_impacts` method.

In [22]:
supchain.calc_production_impacts(direct_impact_usa, exp_usa, impacted_secs=impacted_secs, io_approach='ghosh')

/Users/aciullo/Documents/GitHub/climada_petals/climada_petals/engine/supplychain.py:450: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.secs_stock_exp.loc[:, (mriot_reg_name, impacted_secs)] += (
/Users/aciullo/Documents/GitHub/climada_petals/climada_petals/engine/supplychain.py:453: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.secs_stock_imp.loc[:, (mriot_reg_name, impacted_secs)] += (


## Calculate expected production losses

In [27]:
supchain.calc_production_eai(direct_impact_usa.frequency)

In [28]:
supchain.dir_prod_impt_eai.loc[('USA', impacted_secs)]

region  sector                                                                                                                                             
USA     Construction                                                                                                                                           5.658314e+08
        Wholesale and retail trade and repair of motor vehicles and motorcycles                                                                                1.799712e+08
        Wholesale trade, except of motor vehicles and motorcycles                                                                                              7.617599e+08
        Retail trade, except of motor vehicles and motorcycles                                                                                                 6.487859e+08
        Land transport and transport via pipelines                                                                                                          

In [25]:
supchain.indir_prod_impt_eai.loc[('CHE', slice(None))]

sector
Crop and animal production, hunting and related service activities                                                                                     4.649573e+04
Forestry and logging                                                                                                                                   2.538937e+03
Fishing and aquaculture                                                                                                                                7.759029e+01
Mining and quarrying                                                                                                                                   1.018396e+04
Manufacture of food products, beverages and tobacco products                                                                                           1.916288e+05
Manufacture of textiles, wearing apparel and leather products                                                                                          1.948900e+04
Manufactu

In [26]:
supchain.tot_prod_impt_eai.loc[('CHE', slice(None))]

sector
Crop and animal production, hunting and related service activities                                                                                     4.649573e+04
Forestry and logging                                                                                                                                   2.538937e+03
Fishing and aquaculture                                                                                                                                7.759029e+01
Mining and quarrying                                                                                                                                   1.018396e+04
Manufacture of food products, beverages and tobacco products                                                                                           1.916288e+05
Manufacture of textiles, wearing apparel and leather products                                                                                          1.948900e+04
Manufactu